In [16]:
import openai
openai.api_key = open("../key.txt","r").read().strip('\n')

In [17]:
completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages = [{"role": "user", "content": "How can i use Chatgpt-api"}]
)

print(completion)

{
  "id": "chatcmpl-8eOJf2FFCjgR8OcsA79e7XQQcyW5m",
  "object": "chat.completion",
  "created": 1704637283,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To use the ChatGPT API, you'll need to do the following:\n\n1. OpenAI provides access to the GPT-3 model through its API. You'll need to sign up at beta.openai.com/signup to get access to the API. \n\n2. Once you have access, you can use it via an HTTP API. You should make a POST request to `https://api.openai.com/v1/engines/davinci-codex/completions`. The body of the request should contain your prompt and specific parameters you want to set. For example:\n\n```json\n{\n  \"model\": \"text-davinci-002\",\n  \"prompt\": \"Translate English to French: {your text here}\",\n  \"max_tokens\": 60\n}\n```\n\n3. You need to include an `Authorization` header with your API key: \n\n```json\n{\n  'Authorization': 'Bearer YOUR_API_KEY',\n  'Content-Type': 'appli

In [18]:
reply = completion['choices'][0].message.content
print(reply)

To use the ChatGPT API, you'll need to do the following:

1. OpenAI provides access to the GPT-3 model through its API. You'll need to sign up at beta.openai.com/signup to get access to the API. 

2. Once you have access, you can use it via an HTTP API. You should make a POST request to `https://api.openai.com/v1/engines/davinci-codex/completions`. The body of the request should contain your prompt and specific parameters you want to set. For example:

```json
{
  "model": "text-davinci-002",
  "prompt": "Translate English to French: {your text here}",
  "max_tokens": 60
}
```

3. You need to include an `Authorization` header with your API key: 

```json
{
  'Authorization': 'Bearer YOUR_API_KEY',
  'Content-Type': 'application/json'
}
```
4. The response to this request will contain the model's completion in the field `.choices[0].text`.

Please note that it costs money per token to use this service, both for the input and output tokens.

For more information, you can also refer to th

In [19]:
!pip install python-dotenv
!pip install matplotlib
!pip install pydantic
!pip install pyecharts
!pip install fastapi

In [20]:
#PLOT GPT generated top station data
import re
import matplotlib.pyplot as plt
from pyecharts.charts import Bar, Page, Pie, Grid
from pyecharts import options as opts



# Define the function to plot the gpt data
def plot_gpt_data_to_html(filtered_gpt_data):
    data = filtered_gpt_data

    # Updated pattern to handle both data formats
    pattern = r'(\w[\w\s,]*[^\s])\s\((\d+)\)|\(([\w\s,]+), (\d+)\)'
    matches = re.findall(pattern, data)

    # Extracting station names and frequencies considering both matching groups from the updated pattern
    stations = [match[0] if match[2] == '' else match[2] for match in matches]
    frequencies = [int(match[1]) if match[3] == '' else int(match[3]) for match in matches]

    bar_chart = (
        Bar()
        .add_xaxis(stations)
        .add_yaxis("Frequenz", frequencies, tooltip_opts=opts.TooltipOpts(is_show=True, trigger="axis", axis_pointer_type="cross"))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Betroffene Dauer"),  
            xaxis_opts=opts.AxisOpts(
                name="Stationen",  # X-Axis title
                axislabel_opts=opts.LabelOpts(
                    rotate=45,   # Try 45-degree rotation
                    font_size=8  # Further reduce font size
                )
            ),
            yaxis_opts=opts.AxisOpts(
                name="Frequenz",  # Y-Axis title
            )
        )
    )

    return bar_chart.render_notebook()



# Define the function to plot the line data
def plot_line_data_to_html(lines_data_list):
    data = lines_data_list
    page = Page(layout=Page.SimplePageLayout)
    bars = []

    for idx, week_data in enumerate(data, 1):
        labels, values = zip(*week_data)
        bar_chart = (
            Bar()
            .add_xaxis(list(labels))
            .add_yaxis(f"Woche {idx}", list(values), label_opts=opts.LabelOpts(is_show=False))
            .set_global_opts(
                title_opts=opts.TitleOpts(title=f"Woche {idx} Vorfälle je Linie"),
                xaxis_opts=opts.AxisOpts(
                    name="Linie",  # X-Axis title
                    axislabel_opts=opts.LabelOpts(rotate=45)  # Optional: Rotate x-axis labels if needed
                ),
                yaxis_opts=opts.AxisOpts(
                    name="Anzahl Vorfälle"  # Y-Axis title
                )
            )
        )
        bars.append(bar_chart)

    page.add(*bars)
    
    print("Generating plot...")
    # Return the embeddable HTML
    #return page.render_embed()
    return page.render_notebook()

# Define the function to plot the accident data
def plot_accident_data_to_html(accident_data_list):
    data = accident_data_list

    def generate_pie(data_item, title):
        categories = list(data_item.keys())
        values = list(data_item.values())
        pie = (
            Pie()
            .add("", [list(z) for z in zip(categories, values)])
            .set_global_opts(title_opts=opts.TitleOpts(title=title), legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="85%"))
            .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))  # Displaying category name and percentage
        )
        return pie

    page = Page()
    for index, data_item in enumerate(data):
        page.add(generate_pie(data_item, f"Woche {index+1}"))
    
    

        
    # Return the embeddable HTML
    #return page.render_embed()
    return page.render_notebook()


In [472]:
tem = 1

In [473]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=tem,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" 
                  + resulted_data 
                  + "Im Datenkontext der Wiener-Linie sind unter (title) betroffene Linien unter (start) betroffene Startzeit und unter (end) betroffene Endzeit verzeichnet. Welche 10 Linien sind am häufigsten betroffen? Wie lange ist die insgesamt betroffene Zeit, die jede dieser 10 verzögerten Linien？Geben Sie nur in diesem Format aus: 1. (Linien, Gesamtzahl der Vorfälle, insgesamt betroffene Zeit in Format Stunden Minuten Sekunden). Zum Beispiel: 1. (39A, 2, 5Stunden 24Minuten 32Sekunden)."
       }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data):        
    # Convert the provided timestamp range strings to datetime objects
    ##############################################
    #date_format = "%Y-%m-%d %H:%M:%S"
    #start_datetime = datetime.strptime(start_time, date_format)
    #end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description'],
            'start': entry['start'],
            'end': entry['end']
            
        } 
        for entry in data['incidents'] 
        #if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    #############################################
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data




# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2023-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


##########################################
# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        #filtered_gpt_data = get_incident_data(data, start_str, end_str)
        filtered_gpt_data = get_incident_data(data)
        
        print("Filtered GPT data:")
        print(filtered_gpt_data)
        break
    #return plot_gpt_data_to_html(filtered_gpt_data)
#############################################
# Define the function to read the data
def read_data():
    with open('../hypothesis_2/hypo2-DE_2023-8-15.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


In [474]:
get_gpt_data(get_date_ranges())

Start: 2023-08-15 00:00:00, End: 2023-08-16 00:00:00
[{"title": "D: Fremder Verkehrsunfall", "description": "Wegen eines fremden Verkehrsunfalles im Bereich Zahnradbahnstra\u00c3\u009fe f\u00c3\u00a4hrt die Linie D nur zwischen Absberggasse und Liechtenwerder Platz. Die St\u00c3\u00b6rung dauert voraussichtlich bis 00 Uhr 15 Minuten 00 Sekunden!", "start": "23 Uhr 37 Minuten 31 Sekunden am 15. August 2023", "end": "23 Uhr 59 Minuten 01 Sekunden am 15. August 2023"}, {"title": "42: Falschparker", "description": "Wegen eines Falschparkers im Bereich Kreuzgasse 21 ist die Linie 42 in Fahrtrichtung Schottentor U an der Weiterfahrt gehindert. Die St\u00c3\u00b6rung dauert voraussichtlich bis 19 Uhr 00 Minuten 00 Sekunden!", "start": "18 Uhr 44 Minuten 04 Sekunden am 15. August 2023", "end": "19 Uhr 46 Minuten 02 Sekunden am 15. August 2023"}, {"title": "26: Schadhaftes Fahrzeug", "description": "Die Stra\u00c3\u009fenbahnlinie 26 kann wegen eines schadhaften Fahrzeuges derzeit in beiden Ric